In [3]:
# Read and Download standard MODIS LSP data from MCD12Q2
# LSP rasters are organized by: 2001-2024 for SOS, EOS, LOS, POS

# Filter by QA and numcycles 1
# for sos we obtain: Midgreenup_1, eos : MidGreendown_1, 
# los = MidGreendown_1-MidGreenup_1, pos = (EVI_Minimum_1 + EVI_Amplitude_1) * 0.0001
# band values are days since 1970-01-01 

In [4]:
import ee
import geemap
import os

cloud_project = "ee-jeevankatel3"
try: 
    ee.Initialize(project=cloud_project)
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project=cloud_project)

In [5]:
#Configuration
start_year, start_date = 2001, "2001-01-01"
end_year, end_date = 2024, "2024-12-31"

output_dir = "../Data/Standard_Modis_LSP"
os.makedirs(output_dir, exist_ok=True)
roi = ee.FeatureCollection('projects/ee-jeevankatel3/assets/nepRect')
roi_geometry = roi.geometry()
scale = 500

mcd12q2 = ee.ImageCollection('MODIS/061/MCD12Q2').filterBounds(roi_geometry).filterDate(start_date, end_date)
#mcd12q2

In [6]:
#download function for each metric

def calculate_doy(image, band_name, year):
    # Calculate Jan 1 epoch for the year
    jan1_epoch = ee.Date.fromYMD(year, 1, 1).millis().divide(86400000)
    
    absolute_days = image.select(band_name)
    doy = absolute_days.subtract(jan1_epoch)
    
    # Handle leap years: allow DOY up to 366 to preserve true calendar dates
    is_leap = (year % 4 == 0) and ((year % 100 != 0) or (year % 400 == 0))
    if is_leap:
        max_doy = 366
    else:
        max_doy = 365    
    qa_mask = image.select('QA_Overall_1').lte(1)
    range_mask = doy.gte(0).And(doy.lt(max_doy))
    fill_mask = absolute_days.neq(32767) 
    
    valid_mask = qa_mask.And(range_mask).And(fill_mask)
    
    return doy.updateMask(valid_mask).unmask(-999).toFloat()

def download_sos(image, year):
    sos = calculate_doy(image, 'MidGreenup_1', year).clip(roi_geometry)
    sos_dir = os.path.join(output_dir, "sos")
    os.makedirs(sos_dir, exist_ok=True)
    outpath_full = os.path.join(sos_dir, f"sos_{year}.tif")
    geemap.ee_export_image(sos, filename=outpath_full, region=roi_geometry, crs="EPSG:4326", scale = 500, file_per_band=False)

def download_eos(image, year):
    eos = calculate_doy(image, 'MidGreendown_1', year).clip(roi_geometry)
    eos_dir = os.path.join(output_dir, "eos")
    os.makedirs(eos_dir, exist_ok=True)
    outpath_full = os.path.join(eos_dir, f"eos_{year}.tif")
    geemap.ee_export_image(eos, filename=outpath_full, region=roi_geometry, crs="EPSG:4326", scale = 500, file_per_band=False)

def download_pos(image, year):
    #calculate pos
    pos = image.select('EVI_Amplitude_1').add(image.select('EVI_Minimum_1')).multiply(0.0001).clip(roi_geometry)

    #filter for range (0 to 1)
    range_mask = pos.gte(0).And(pos.lte(1))
    #put qa filter
    qa_mask = image.select('QA_Overall_1').lte(1)

    #apply mask and clip
    valid_mask = qa_mask.And(range_mask)
    pos = pos.updateMask(valid_mask).unmask(-999).clip(roi_geometry)

    #export
    pos_dir = os.path.join(output_dir, "pos")
    os.makedirs(pos_dir, exist_ok=True)
    outpath_full = os.path.join(pos_dir, f"pos_{year}.tif")
    geemap.ee_export_image(pos, filename=outpath_full, region=roi_geometry, crs="EPSG:4326", scale = 500, file_per_band=False)

def download_los(image, year):

    is_leap = (year % 4 == 0) and ((year % 100 != 0) or (year % 400 == 0))
    days_in_year = 366 if is_leap else 365

    diff = image.select('MidGreendown_1').subtract(image.select('MidGreenup_1'))
    
    los = diff.where(diff.lt(0), diff.add(days_in_year))
    
    qa_mask = image.select('QA_Overall_1').lte(1)
   
    fill_mask = image.select('MidGreendown_1').neq(32767) \
        .And(image.select('MidGreenup_1').neq(32767))

    range_mask = los.gt(0).And(los.lte(366))

    valid_mask = qa_mask.And(fill_mask).And(range_mask)

    final_los = los.updateMask(valid_mask).unmask(-999).clip(roi_geometry)
   
    los_dir = os.path.join(output_dir, "los")
    os.makedirs(los_dir, exist_ok=True)
    outpath_full = os.path.join(los_dir, f"los_{year}.tif")
    
    geemap.ee_export_image(
        final_los, 
        filename=outpath_full, 
        region=roi_geometry, 
        crs="EPSG:4326", 
        scale=500, 
        file_per_band=False
    )

In [7]:
for year in range(start_year, end_year + 1):
    current_year = f"{year}"+ "_01_01"
    #filter for year
    filtered_image = mcd12q2.filter(ee.Filter.eq('system:index', current_year)).first()
    #filter for pixels with numseasons = 1
    filtered_image = filtered_image.updateMask(filtered_image.select('NumCycles').eq(1))
    download_sos(filtered_image, year)
    download_eos(filtered_image, year)
    download_los(filtered_image, year)
    download_pos(filtered_image, year)
    print("Download Completed for:", current_year)

Generating URL ...
Please wait ...
Data downloaded to c:\Users\A S U S\Documents\Study\Land Surface Phenology\LSP_WS\Data\Standard_Modis_LSP\sos\sos_2001.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\A S U S\Documents\Study\Land Surface Phenology\LSP_WS\Data\Standard_Modis_LSP\eos\eos_2001.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\A S U S\Documents\Study\Land Surface Phenology\LSP_WS\Data\Standard_Modis_LSP\los\los_2001.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\A S U S\Documents\Study\Land Surface Phenology\LSP_WS\Data\Standard_Modis_LSP\pos\pos_2001.tif
Download Completed for: 2001_01_01
Generating URL ...
Please wait ...
Data downloaded to c:\Users\A S U S\Documents\Study\Land Surface Phenology\LSP_WS\Data\Standard_Modis_LSP\sos\sos_2002.tif
Generating URL ...
Please wait ...
Data downloaded to c:\Users\A S U S\Documents\Study\Land Surface Phenology\LSP_WS\Data\Standard_Modis_LSP\eos\eos_2002.tif
Generating URL ..